## Custom Image Classification with PyTorch

This notebook demonstrates the process of building a custom image classification model using PyTorch. The notebook includes steps for setting up the data pipeline, defining a custom dataset, applying transformations, and training a convolutional neural network (CNN) model. Specifically, it utilizes datasets from Places365 and CoastSnap.
By the end of this notebook, you will have a trained model capable of classifying images from coasts and non-coasts.

In [1]:
import os
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Dataset and Directory Structure

In this notebook, we utilize two datasets: **Places365** and **CoastSnap** from the coastline dataset. You can download the dataset from [Places365](http://places2.csail.mit.edu/download.html).

The datasets are stored in the `data` directory, with subdirectories for each dataset:
- `data/Places365`: Contains images for the Places365 dataset.
- `data/CoastSnap`: Contains images for the CoastSnap dataset.

These directories will be referenced in the notebook for data loading and processing.

In [3]:
data_dir = 'data'
places_dir = os.path.join(data_dir, 'Places365')
coastsnap_dir = os.path.join(data_dir, 'CoastSnap')

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [6]:
def get_image_paths_and_labels(base_dir, label):
    img_paths = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_paths.append(os.path.join(root, file))
    labels = [label] * len(img_paths)
    return img_paths, labels

In [7]:
places_paths, places_labels = get_image_paths_and_labels(places_dir, 0)
coastsnap_paths, coastsnap_labels = get_image_paths_and_labels(coastsnap_dir, 1)

In [8]:
img_paths = places_paths + coastsnap_paths
labels = places_labels + coastsnap_labels

train_paths, val_paths, train_labels, val_labels = train_test_split(img_paths, labels, test_size=0.2, stratify=labels, random_state=42)

In [9]:
train_dataset = CustomDataset(train_paths, train_labels, transform=transform)
val_dataset = CustomDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [10]:
print('Train dataset size:', len(train_dataset))
print('Validation dataset size:', len(val_dataset))

Train dataset size: 3453
Validation dataset size: 864


In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

In [12]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train and validate the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward propagation
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward propagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate loss and accuracy
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / total
    train_accuracy = 100 * correct / total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss = val_loss / val_total
    val_accuracy = 100 * val_correct / val_total

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# Save the model
torch.save(model.state_dict(), 'coast_classifier.pth')

Epoch [1/10], Train Loss: 0.1365, Train Accuracy: 95.71%, Val Loss: 2.4182, Val Accuracy: 66.44%
Epoch [2/10], Train Loss: 0.0598, Train Accuracy: 97.94%, Val Loss: 0.1114, Val Accuracy: 96.88%
Epoch [3/10], Train Loss: 0.0405, Train Accuracy: 98.47%, Val Loss: 0.0529, Val Accuracy: 98.61%
Epoch [4/10], Train Loss: 0.0456, Train Accuracy: 98.47%, Val Loss: 0.0298, Val Accuracy: 98.84%
Epoch [5/10], Train Loss: 0.0168, Train Accuracy: 99.45%, Val Loss: 0.1570, Val Accuracy: 94.21%
Epoch [6/10], Train Loss: 0.0546, Train Accuracy: 98.26%, Val Loss: 0.0718, Val Accuracy: 97.34%
Epoch [7/10], Train Loss: 0.0340, Train Accuracy: 99.13%, Val Loss: 0.0286, Val Accuracy: 98.96%
Epoch [8/10], Train Loss: 0.0215, Train Accuracy: 99.30%, Val Loss: 0.0450, Val Accuracy: 98.26%
Epoch [9/10], Train Loss: 0.0244, Train Accuracy: 99.30%, Val Loss: 0.0356, Val Accuracy: 98.73%
Epoch [10/10], Train Loss: 0.0114, Train Accuracy: 99.65%, Val Loss: 0.0047, Val Accuracy: 99.88%


In [4]:
import torch
from torch import nn
from torchvision import transforms, models
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  
model.load_state_dict(torch.load('coast_classifier.pth', map_location=device))
model = model.to(device)
model.eval()

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

def predict_image(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        return predicted.item()

image_path = 'Songs_crimsonthrone.jpg'

prediction = predict_image(image_path)


if prediction == 1:
    print(f'The image {image_path} is classified as a coast.')
else:
    print(f'The image {image_path} is classified as non-coastal.')

The image Songs_crimsonthrone.jpg is classified as non-coastal.
